In [31]:
import re

re_art  = re.compile(r"@article\s\{.*\n(\t.*\n)+^\}", re.MULTILINE)
re_proc = re.compile(r"@proceedings\s\{.*\n(\t.*\n)+^\}", re.MULTILINE)
articles = ""
proceedings = ""
book = ""
with open("ieor-article.bib", 'r') as f:
    articles = f.read()

with open("ieor-proceedings.bib", 'r') as f:
    proceedings = f.read()

# with open("ieor-book.bib", 'r') as f:
#     books = f.read()  

In [38]:
import pandas as pd

pub = pd.DataFrame(columns=["id", "title", "journal", "year", "authors", "kind"])

def add(pub, kind, regex, string):
    recomp = re.compile(regex, re.MULTILINE)
    for ar in re.finditer(recomp, string):
        art = ar.group(0)
        title = re.findall(r'\ttitle\s*=\s*\{(.*)\}', art); title = title[0] if len(title)>0 else pd.NA
        journal = re.findall(r'\tjournal\s*=\s*\{(.*)\}',art); journal = journal[0] if len(journal)>0 else pd.NA
        auth  = re.findall(r'\tauthor\s*=\s*\{(.*)\}',art); auth = auth[0] if len(auth)>0 else pd.NA
        year = re.findall(r'\tyear\s*=\s*\{(.*)\}',art); year = year[0] if len(year)>0 else pd.NA

        pub.loc[len(pub.index)] = [len(pub.index), title, journal, year, auth, kind]
    return pub

pub = add(pub, "article", r"@article\s\{.*\n(\t.*\n)+^\}", articles)
pub = add(pub, "proceedings", r"@proceedings\s\{.*\n(\t.*\n)+^\}",  proceedings)
# pub = add(pub, "inbook", r"@inbook\s\{.*\n(\t.*\n)+^\}",  books)
# pub = add(pub, "booklet", r"@booklet\s\{.*\n(\t.*\n)+^\}",  books)
# pub = add(pub, "book", r"@book\s\{.*\n(\t.*\n)+^\}",  books)

In [37]:
pub

,id,title,journal,year,authors,kind
0,0,Almost exact risk budgeting with return foreca...,Operations Research Letters,2023,Avinash Bhardwaj and Manjesh Kumar Hanawal and...,article
1,1,Approximate solutions to constrained risk-sens...,European Journal of Operational Research,2023,Uday M Kumar and Sanjay P Bhat and Veeraruna K...,article
2,2,Dynamics of solar energy entrepreneurship in r...,Energy for Sustainable Development,2023,Rajat Mishra and Sayli Shiradkar and Kelsey We...,article
3,3,An OR Approach for WORdle,Informs Transactions on Education,2023,Simran Lakhani and Mustafa Vora and Ashutosh M...,article
4,4,Choice of Intraoperative Ultrasound adjuncts f...,BMC Med Inform Decis Mak,2022,Manoj Kumar and Santosh noronha and Narayan Ra...,article
...,...,...,...,...,...,...
481,481,Non-stationary models of manufacturing systems...,IEEE International Symposium on Intelligent Co...,1995,Y Narahari and Nandyala Hemachandra,proceeding
482,482,Crew Scheduling for urban bus operations,ORSI Annual Convention,1993,"Suresh, J.R and Narayan Rangaraj",proceeding
483,483,Network based pricing for passenger services o...,National Conference on Transportation Systems ...,1993,G Raghuram and Narayan Rangaraj and M.V Kamath,proceeding
484,484,A systematic and quantitative view of capacity...,ORSI Annual Convention,1993,"Narayan Rangaraj and Paes, E",proceeding


In [39]:
# 19 books 
# 487 articles + conferences 